# Analysis of agent-based model

M. Tsvetkova, **The effects of reputation on inequality in network cooperation games**, *Phil. Trans. B* (2021).

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import matplotlib.patches as mpatches
import seaborn as sb
import random
from matplotlib import cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr

import os
import sys

module_path = os.path.abspath(os.path.join('../modules/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from ineq import *
from plot import *

NETS = ['random rewiring', 'strategic updating']
NET_DIC = {'rewired-random': 'random rewiring', 'rewired-clustered': 'random rewiring', 
           'strategic-random': 'strategic updating', 'strategic-clustered': 'strategic updating'}

REPUT = [0, 1]
COLORS = {'reputation = 0': '1', 'reputation = 1': 'red', 'reputation = 3': 'red'}
ORDER_SCATTER = [('random rewiring', 'reputation = 0'), ('random rewiring', 'reputation = '+str(REPUT[0])),
                ('strategic updating', 'reputation = 0'), ('strategic updating', 'reputation = '+str(REPUT[1]))]


SIMFILE = 'ineq_rep_model ineq_rep013_nei25-table.csv' #'ineq_rep_model 100_var_types-table.csv' 


### 1. Define data functions

In [76]:
def get_sim_data(dirname, payoff_fnc=None, nei=None):   
    """Read csv file with simulation data and return dictionaries
    with Gini coefficient and average cooperation in last period.
    """
    
    data = pd.read_csv(dirname, header=6)    
    if payoff_fnc != None:
        data = data[data['payoff-function'] == payoff_fnc]
    if nei != None:
        data = data[data['avg-partners'] == nei]
        
    data = data[['network', 'reputation', 'percent-defectors', 'percent-altruists',
                 '(gini-index-reserve / population-size) * 2', 'mean [action-choice] of turtles']]
    data.columns = ['network', 'reputation', 'percent-defectors', 'percent-altruists', 'gini', 'cooperation']
    data['network'] = [NET_DIC[i.strip('"')] for i in data['network']]
    
    ineq = {n: {r: [] for r in REPUT} for n in NETS}
    coop = {n: {r: [] for r in REPUT} for n in NETS}
    for n in NETS:
        for r in REPUT:
            ineq[n][r] = [[np.mean(get_subdat(data, n, r, d, a)['gini'].values) \
                               for a in range(0, 26, 1)] \
                               for d in range(0, 26, 1)]
            coop[n][r] = [[np.mean(get_subdat(data, n, r, d, a)['cooperation'].values) \
                               for a in range(0, 26, 1)] \
                               for d in range(0, 26, 1)]

    return ineq, coop


def get_sim_corr(dirname, payoff_fnc=None, nei=None):   
    """Read csv file with simulation data and return dictionaries
    with correlations between final payoff and average cooperation.
    """
    
    data = pd.read_csv(dirname, header=6)    
    if payoff_fnc != None:
        data = data[data['payoff-function'] == payoff_fnc]
    if nei != None:
        data = data[data['avg-partners'] == nei]
        
    data = data[['network', 'reputation', 'percent-defectors', 'percent-altruists',
                 '[run number]', '[mean action-history] of turtles', '[wealth] of turtles']]
    data.columns = ['network', 'reputation', 'percent-defectors', 'percent-altruists',
                    'run', 'cooperativeness', 'wealth']
    data['network'] = [NET_DIC[i.strip('"')] for i in data['network']]
    
    corr = {n: {r: [] for r in REPUT} for n in NETS}
    for n in NETS:
        for r in REPUT:
            corr[n][r] = [[get_mean_corr(data, n, r, d, a) for a in range(0, 26, 1)] \
                               for d in range(0, 26, 1)]

    return corr


def get_subdat(data, n, r, d, a):
    return data[(data['network']==n) & (data['reputation']==r) & \
                 (data['percent-defectors']==d) & (data['percent-altruists']==a)]


def get_mean_corr(data, n, r, d, a):
    df = get_subdat(data, n, r, d, a)
    xy = zip(df['cooperativeness'].values, df['wealth'].values)
    return np.mean([pearsonr(get_vals_from_str(x), get_vals_from_str(y))[0] for x, y in xy])
    

def get_vals_from_str(datum):
    return [float(i) for i in datum.rstrip(']').lstrip('[').split()]


def get_sim_example(dirname):   
    """Read csv file with simulation data, pick an example run for
    reputation = 0 and reputation = 3, and return dictionary with 
    (run_number, reputation) as key and (avg_coopertion, wealth)
    for all agents in final period as value.
    ."""
    
    data = pd.read_csv(dirname, header=6)    
    data = data[['network', 'reputation', 'percent-defectors', 
                  'percent-altruists', '[run number]', '[mean action-history] of turtles', '[wealth] of turtles']]
    data.columns = ['network', 'reputation', 'percent-defectors', 
                    'percent-altruists', 'run', 'cooperativeness', 'wealth']
    data['network'] = [NET_DIC[i.strip('"')] for i in data['network']]
    
    dic = {}
    for n in NETS:
        for r in REPUT:
            # Restrict to defectors = 20% and altruists = 15%
            subdata = get_subdat(data, n, r, d=20, a=15)
            run = random.sample(list(subdata['run'].unique()), k=1)[0]
            example = subdata[subdata['run']==run]
            coops = get_vals_from_str(example['cooperativeness'].values[0]) 
            wealths = get_vals_from_str(example['wealth'].values[0]) 
            dic[(n, 'reputation = ' + str(r))] = (coops, wealths)

    return dic


### 2. Define plotting functions

In [77]:
def custom_axlim(ax, ifylabel=True, ylabel='Gini coefficient', ymax=0.75):
    ax.set_yticks( [i/10 for i in range(int(10*ymax)+1)] )
    ax.set_ylim(0, ymax)
    ax.set_xlim(0.5, 7.1)
    if ifylabel: 
        ax.set_ylabel(ylabel, fontsize=6)
    else:
        ax.set_yticklabels(['             '+str(round(i, 1)) for i in np.linspace(0, ymax-0.05, num = int(ymax/0.1)+1)])


def get_legend_handles():
    rep0 = mpatches.Patch(color='1', label='reputation = 0', ec='k', lw=0.25)
    rep = mpatches.Patch(color='red', label='reputation = ' + str(REPUT[1]), ec='k', lw=0.25)
    return [rep0, rep]


### 3. Plot boxplots for average cooperation and inequality

In [78]:
X = np.arange(0, 26, 1)
Y = np.arange(0, 26, 1)
X, Y = np.meshgrid(X, Y)

def plot_surfaces(data, net_update, z_title, z_vals, title, fname):
    fig = go.Figure()
    fig.add_trace(
        go.Surface(x=X, y=Y, z=np.array(data[net_update][REPUT[0]]), colorscale='Greys', showscale=False))
    fig.add_trace(
        go.Surface(x=X, y=Y, z=np.array(data[net_update][REPUT[1]]), colorscale='Reds', opacity=0.95, showscale=False))
    
    camera = dict(
        eye=dict(x=-1.5, y=-2, z=1),
        center=dict(x=0, y=0, z=0)
    )

    fig.update_layout(
        scene = dict(
            xaxis = dict(range=[0, 25]),
            yaxis = dict(range=[0, 25]),
            zaxis = dict(range=[2*z_vals[0]-z_vals[1], z_vals[-1]], tickvals=z_vals),
            xaxis_title='% altruists',
            yaxis_title='% defectors',
            zaxis_title=z_title,
            aspectratio=dict(x=1, y=1, z=0.5)),
        scene_camera=camera,
        margin=dict(r=0, l=0, b=0, t=0, pad=0),
        height=500,
        width=800, autosize=False    
    )
    fig.show()
    fig.write_image("../plots/" + fname + ".pdf") #, engine="orca"

# Get data
sim_gini, sim_coop = get_sim_data(SIMFILE, payoff_fnc="average", nei=5) #per neighbor, average
# Plot separately to avoid formatting issues
plot_surfaces(sim_coop, 'random rewiring', 'Avg. cooperation', [0.2, 0.4, 0.6, 0.8, 1.0], 'Random rewiring', 'sim5_rep1_coop_rr_avg')
plot_surfaces(sim_coop, 'strategic updating', 'Avg. cooperation', [0.2, 0.4, 0.6, 0.8, 1.0], 'Strategic updating', 'sim5_rep1_coop_su_avg')
plot_surfaces(sim_gini, 'random rewiring', 'Gini coefficient', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6], 'Random rewiring', 'sim5_rep1_gini_rr_avg')
plot_surfaces(sim_gini, 'strategic updating', 'Gini coefficient', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6], 'Strategic updating', 'sim5_rep1_gini_su_avg')


### 4. Plot an example scatter plot for wealth vs. cooperation

In [79]:
'''# Set random seed to 2 replicate plots for paper
random.seed(2) 

# Initialize figure and axes
init_plot()
f = plt.figure(figsize=(6, 1.6))
f, axes = plt.subplots(1, 4, figsize=(6, 1.6), dpi=150)

# Get data and plot
cws = get_sim_example(SIMFILE)
for p in range(4):
    data = cws[ORDER_SCATTER[p]]
    axes[p].scatter(data[0], data[1], \
                    color=COLORS[ORDER_SCATTER[p][1]], marker='o', linewidths=0.25, s=10, edgecolors='k')
    print(pearsonr(data[0], data[1]))
    axes[p].set_xlim(-0.075, 1.075)
    axes[p].set_ylim(-50, 4050)
    axes[p].set_title(ORDER_SCATTER[p][1])
    axes[p].set_xlabel('Cooperation')
    if p==0: 
        axes[p].set_ylabel('Wealth')
    else:
        axes[p].set_yticklabels([])

# Format figure and save
plt.suptitle('A) Random rewiring                                                                    B) Strategic updating'\
             , y=1.1, x=0.53)
plt.tight_layout(pad=1, w_pad=3, h_pad=0)
plt.show()
f.savefig('../plots/fig_2.pdf', format='pdf', bbox_inches='tight')    
'''

"# Set random seed to 2 replicate plots for paper\nrandom.seed(2) \n\n# Initialize figure and axes\ninit_plot()\nf = plt.figure(figsize=(6, 1.6))\nf, axes = plt.subplots(1, 4, figsize=(6, 1.6), dpi=150)\n\n# Get data and plot\ncws = get_sim_example(SIMFILE)\nfor p in range(4):\n    data = cws[ORDER_SCATTER[p]]\n    axes[p].scatter(data[0], data[1],                     color=COLORS[ORDER_SCATTER[p][1]], marker='o', linewidths=0.25, s=10, edgecolors='k')\n    print(pearsonr(data[0], data[1]))\n    axes[p].set_xlim(-0.075, 1.075)\n    axes[p].set_ylim(-50, 4050)\n    axes[p].set_title(ORDER_SCATTER[p][1])\n    axes[p].set_xlabel('Cooperation')\n    if p==0: \n        axes[p].set_ylabel('Wealth')\n    else:\n        axes[p].set_yticklabels([])\n\n# Format figure and save\nplt.suptitle('A) Random rewiring                                                                    B) Strategic updating'             , y=1.1, x=0.53)\nplt.tight_layout(pad=1, w_pad=3, h_pad=0)\nplt.show()\nf.savefig('..

In [80]:
# Get data
sim_corr = get_sim_corr(SIMFILE, payoff_fnc="average", nei=5)
# Plot separately to avoid formatting issues
plot_surfaces(sim_corr, 'random rewiring',  'Pearson correlation', [-0.5, 0, 0.5, 1], 'Random rewiring', 'sim5_rep1_corr_rr_avg')
plot_surfaces(sim_corr, 'strategic updating', 'Pearson correlation', [-0.5, 0, 0.5, 1], 'Strategic updating', 'sim5_rep1_corr_su_avg')
